### Shoreline change analysis code ###

<p>This uses the detected shorelines from the initial steps in coastsat</p>

In [6]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from matplotlib import gridspec
plt.ion()
import pandas as pd
from datetime import datetime
from coastsat import SDS_download, SDS_preprocess, SDS_shoreline, SDS_tools, SDS_transects

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### initialize project ###

<p> Setup settings and load shoreline points </p>

In [3]:
# init setttings
# region of interest (longitude, latitude)
polygon = [[
  [123.28654048, 13.89552683],
  [123.28654048, 13.90364783],
  [123.28977272, 13.90364783],
  [123.28977272, 13.89552683],
  [123.28654048, 13.89552683]
]]

# it's recommended to convert the polygon to the smallest rectangle (sides parallel to coordinate axes)       
polygon = SDS_tools.smallest_rectangle(polygon)
# date range
dates = ['1990-01-01', '2022-10-16']
# satellite missions ['L5','L7','L8','L9','S2']
sat_list = ['L5', 'L7', 'S2']
# choose Landsat collection 'C01' or 'C02'
collection = 'C01'
# name of the site
sitename = 'CATIM'
# directory where the data will be stored
filepath = os.path.join(os.getcwd(), 'data')
# put all the inputs into a dictionnary
inputs = {'polygon': polygon, 'dates': dates, 'sat_list': sat_list, 'sitename': sitename, 'filepath':filepath,
         'landsat_collection': collection}

# before downloading the images, check how many images are available for your inputs
# SDS_download.check_images_available(inputs);

settings = { 
    # general parameters:
    'cloud_thresh': 0.5,        # threshold on maximum cloud cover
    'dist_clouds': 100,         # ditance around clouds where shoreline can't be mapped
    'output_epsg': 3124,       # epsg code of spatial reference system desired for the output
    # quality control:
    'check_detection': False,    # if True, shows each shoreline detection to the user for validation
    'adjust_detection': False,  # if True, allows user to adjust the postion of each shoreline by changing the threhold
    'save_figure': True,        # if True, saves a figure showing the mapped shoreline for each image
    # [ONLY FOR ADVANCED USERS] shoreline detection parameters:
    'min_beach_area': 2000,     # minimum area (in metres^2) for an object to be labelled as a beach
    'min_length_sl': 500,       # minimum length (in metres) of shoreline perimeter to be valid
    'cloud_mask_issue': False,  # switch this parameter to True if sand pixels are masked (in black) on many images  
    'sand_color': 'default',    # 'default', 'latest', 'dark' (for grey/black sand beaches) or 'bright' (for white sand beaches)
    'pan_off': False,           # True to switch pansharpening off for Landsat 7/8/9 imagery
    # add the inputs defined previously
    'inputs': inputs,
}



In [3]:
# load shoreline points
filepath = os.path.join(inputs['filepath'], sitename)
with open(os.path.join(filepath, sitename + '_output' + '.pkl'), 'rb') as f:
    output = pickle.load(f)
# remove duplicates (images taken on the same date by the same satellite)
output = SDS_tools.remove_duplicates(output)
# remove inaccurate georeferencing (set threshold to 10 m)
output = SDS_tools.remove_inaccurate_georef(output, 10)

222 duplicates
9 bad georef


In [4]:
# load transects
geojson_file = os.path.join(os.getcwd(), 'cagliliog2.geojson')
transects = SDS_tools.transects_from_geojson(geojson_file)

392 transects have been loaded


### calculate intersection points ###

In [5]:
# calculate intersection points
settings_transects = { # parameters for computing intersections
                      'along_dist':          25,        # along-shore distance to use for computing the intersection
                      'min_points':          3,         # minimum number of shoreline points to calculate an intersection # dati 3
                      'max_std':             15,        # max std for points around transect
                      'max_range':           20,        # max range for points around transect # dati 30
                      'min_chainage':        0,      # largest negative value along transect (landwards of transect origin)
                      'multiple_inter':      'auto',    # mode for removing outliers ('auto', 'nan', 'max')
                      'prc_multiple':         0.1,      # percentage of the time that multiple intersects are present to use the max
                     }
cross_distance= SDS_transects.compute_intersection_QC(output, transects, settings_transects) 

# save a .csv file for Excel users
out_dict = dict([])
out_dict['dates'] = output['dates']
for key in transects.keys():
    out_dict[key] = cross_distance[key]
df = pd.DataFrame(out_dict)
fn = os.path.join(settings['inputs']['filepath'],settings['inputs']['sitename'],
                  'ts.csv')
df.to_csv(fn, sep=',')
print('Time-series of the shoreline change along the transects saved as:\n%s'%fn)

### tidal correction ###

In [8]:
# load the measured tide data
filepath = os.path.join(os.getcwd(),'cagliliog_tides.csv')
tide_data = pd.read_csv(filepath, parse_dates=['dates'])
dates_ts = [_.to_pydatetime() for _ in tide_data['dates']]
tides_ts = np.array(tide_data['tide'])

# get tide levels corresponding to the time of image acquisition
dates_sat = output['dates']
tides_sat = SDS_tools.get_closest_datapoint(dates_sat, dates_ts, tides_ts)

# tidal correction along each transect
reference_elevation = 0 # elevation at which you would like the shoreline time-series to be
beach_slope = 1.990542
cross_distance_tidally_corrected = {}
for key in cross_distance_tidally_corrected.keys():
    correction = (tides_sat-reference_elevation)/beach_slope
    cross_distance_tidally_corrected[key] = cross_distance_tidally_corrected[key] + correction
    
# store the tidally-corrected time-series in a .csv file
out_dict = dict([])
out_dict['dates'] = dates_sat
for key in cross_distance_tidally_corrected.keys():
    out_dict[key] = cross_distance_tidally_corrected[key]
df = pd.DataFrame(out_dict)
fn = os.path.join(settings['inputs']['filepath'],settings['inputs']['sitename'],
                  'ts_tidally_corrected.csv')
df.to_csv(fn, sep=',')
print('Tidally-corrected time-series of the shoreline change along the transects saved as:\n%s'%fn)

Extracting closest points: 100%Tidally-corrected time-series of the shoreline change along the transects saved as:
/home/chester/Documents/Thesis/Cagliliog2/CoastSat/data/CATIM/ts_tidally_corrected.csv


In [5]:
fn = os.path.join(settings['inputs']['filepath'],settings['inputs']['sitename'],
                  'ts_.csv')
ts_tdc = pd.read_csv(fn)
ts_tdc

,Unnamed: 0,dates
0,0,1992-06-20 01:28:21+00:00
1,1,1992-07-22 01:28:22+00:00
2,2,1992-08-23 01:27:30+00:00
3,3,1993-06-07 01:27:36+00:00
4,4,1993-11-14 01:27:01+00:00
...,...,...
554,554,2022-09-21 02:23:54+00:00
555,555,2022-09-26 02:23:47+00:00
556,556,2022-10-01 02:23:53+00:00
557,557,2022-10-06 02:23:44+00:00


### despiking ###
<p>this step removes outliers and despikes the intersection points</p>

In [7]:
filepath = os.path.join(os.getcwd(),'data', 'CATIM', 'ts_tidally_corrected.csv')
df = pd.read_csv(filepath, parse_dates=['dates'])
dates = [_.to_pydatetime() for _ in df['dates']]
cross_distance_tidally_corrected = dict([])
for key in transects.keys():
    cross_distance_tidally_corrected[key] = np.array(df[key])

# remove outliers in the time-series (coastal despiking)
settings_outliers = {'max_cross_change':   40,             # maximum cross-shore change observable between consecutive timesteps
                     'otsu_threshold':     [-.5,0],        # min and max intensity threshold use for contouring the shoreline
                     'plot_fig':           False,           # whether to plot the intermediate steps
                    }

cross_distance_despiked = SDS_transects.reject_outliers(cross_distance_tidally_corrected,output,settings_outliers)

KeyError: 'T0'

In [ ]:
# store the tidally-corrected time-series in a .csv file
out_dict = dict([])
out_dict['dates'] = dates_sat
for key in cross_distance_despiked.keys():
    out_dict[key] = cross_distance_despiked[key]
df = pd.DataFrame(out_dict)
fn = os.path.join(settings['inputs']['filepath'],settings['inputs']['sitename'],
                  'ts_despiked.csv')
df.to_csv(fn, sep=',')
print('Despiked time-series of the shoreline change along the transects saved as:\n%s'%fn)

### post processing ###
remove rows where there are large number of missing transect intersects and impute remaining nan values
also preprocess the shorelines csv to match transect time series

In [ ]:
import pandas as pd
import numpy as np

from datetime import datetime

In [ ]:
id_maker = lambda date: date.strftime('%d/%m/%Y')

sls = pd.read_csv('shorelines.csv', parse_dates=['date'])
ts_despiked = pd.read_csv('ts_despiked.csv', index_col=0, parse_dates=['dates'])

sls['id'] = sls.date.apply(id_maker)
ts_despiked['id'] = ts_despiked.dates.apply(id_maker)
ts_despiked['dates_parsed'] = ts_despiked['dates']
ts_despiked['dates'] = ts_despiked['id']

print('ts_despiked: ', ts_despiked.shape)
print('sls: ', sls.shape)

In [ ]:
# drop duplicate rows based on the given ids, since s2 rows come after l7 and l7 before l5, this ensures
# ... we keep last to ensure that the most accurate shoreline position is kept

sls.drop_duplicates(subset=['id'], keep='last', inplace=True)
ts_despiked.drop_duplicates(subset=['id'], keep='last', inplace=True)

print('ts_despiked: ', ts_despiked.shape)
print('sls: ', sls.shape)

### drop nan heavy columns in preprocessed_ts ###

In [ ]:
# drop nan heavy columns in preprocessed_ts
percent = 0.95
total_columns = len(ts_despiked.columns[1:-2])
thresh = int(round(total_columns * 0.95, 0))
ts_despiked_nan_dropped = ts_despiked.dropna(thresh=thresh)

print('''
  delete when {}% of intersects are missing
  which is {} transects.
'''.format(percent, thresh))
print('ts_despiked_nan_dropped shape: ', ts_despiked_nan_dropped.shape)
print('''
  oldest date: {},
  newest date: {}
'''.format(
  min(ts_despiked_nan_dropped.dates_parsed),
  max(ts_despiked_nan_dropped.dates_parsed)
)
)

### fill out remaining nan values ###

In [ ]:
ts_interpolated = ts_despiked_nan_dropped
ts_interpolated = ts_interpolated.set_index(['dates_parsed']).interpolate(method='time', limit_direction='both')

### merge sls and ts_despiked by id ###

In [ ]:
preprocessed_sls = sls[['geoaccuracy', 'id']]
preprocessed_ts = ts_interpolated
ts_sls = preprocessed_ts.set_index('id').join(preprocessed_sls.set_index('id'))


print('sls shape', preprocessed_sls.shape)
print('ts shape', preprocessed_ts.shape)
print('ts_sls shape', preprocessed_ts.shape)

### create new processed transect time series and shorelines ###

In [ ]:
ts_processed = ts_sls[ts_sls.columns[0:398]]

sls_processed = ts_sls[['dates', 'geoaccuracy']]
sls_processed = sls_processed.rename(columns={'dates': 'Date', 'geoaccuracy': 'Uncertainty'})

ts_processed.to_csv('ts_despiked_processed.csv', index=False)
sls_processed.to_csv('shorelines_processed.csv', index=False)

print('''
  ts_processed nan count:
  sls_processed nan count:
'''.format(
  ts_processed.isna().sum().sum(),
  sls_processed.isna().sum().sum()
)